# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [136]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string
from collections import Counter

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
## nltk.download('averaged_perceptron_tagger')
## nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.util import ngrams

## sklearn imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

from collections import Counter

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## 0.1 Load and clean text data

In [4]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [5]:
## subset to one press release and take the string

doj_cut = doj[doj['id'] == '17-1204']['contents']
pharma = doj_cut.to_string(index=False)


### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [6]:
# Part A

# empty string
pharma_clean = ''

# loop to add all alpha and space characters to string 
for character in pharma:
    if character.isalpha() or character.isspace():
        pharma_clean += character

In [7]:
# Part B

# create "bag of words"
words_pharma = word_tokenize(pharma_clean)

# build list of tuples (token, pos)
pos_tags = nltk.pos_tag(words_pharma)

In [9]:
# extract adjectives
adjectives = [word for word, pos in pos_tags if pos.startswith('JJ')] 

# count occurrences of adjectives
adjective_counts = Counter(adjectives)

# subset for 5 most common adjectives
top_adjectives = adjective_counts.most_common(5)

# create a dataframe with the subset data
top_adjectives_df = pd.DataFrame(top_adjectives, columns=['Adjective', 'Count'])


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [10]:
# use nlp to produce doc object
pharma_nlp = nlp(pharma)

# loop over the doc object to extract the tokens
named_entities = [ent.text for ent in pharma_nlp.ents]



In [11]:

# using set() filter and extract unique named entities with the tag "LAW"
unique_law_entities = set([ent.text for ent in pharma_nlp.ents if ent.label_ == "LAW"])


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

In [12]:
# Pharmaceutical companies can face RICO-related investigations and lawsuits if they are accused 
# of engaging in activities that violate the Racketeer Influenced and Corrupt Organizations Act (RICO).

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [13]:
# extract the named entities with the pos tag DATE & use case-insenstive regex to extract entities that contain year/years
date_entities = [ent.text for ent in pharma_nlp.ents if ent.label_ == "DATE" and re.search(r'\b(?:year|years)\b', ent.text, re.I)]

date_entities

['last year', '20 years', 'three years', 'five years', 'three years']

E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [15]:

# Define a regex pattern to find sentences mentioning year lengths
pattern = r'(?:\b\d+\s*(?:year|years)\s*(?:in\s*prison|in\s*custody|probation)?\b)'

# Find all sentence mentions of year lengths
matches = re.findall(pattern, pharma)

matches

['20 years in prison']

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [22]:
# Part A

# Subset the dataframe(press releases)
doj_subset = doj[doj["topics_clean"].isin(["Civil Rights", "Hate Crimes", "Project Safe Childhood"])]

# check shape 
doj_subset.shape

(717, 6)

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [23]:
# Part B

# write function to clean and analyze sentiment of each press release

def remove_named_ent_and_score(press_release):
    # Process the input string using spaCy(nlp)
    doc = nlp(press_release)
    
    # Initialize an empty string to store the cleaned text
    cleaned_text = press_release
    
    # Iterate through named entities and replace them in the cleaned text
    for ent in doc.ents:
        cleaned_text = cleaned_text.replace(ent.text, '')
    
    # initialize SentimentIntensityAnalyzer()
    sentiment_scorer = SentimentIntensityAnalyzer()
    
    # apply SentimentIntensityAnalyzer()
    sentiment = sentiment_scorer.polarity_scores(cleaned_text)
    
    return sentiment



In [25]:
# apply function to dataframe column "contents"
doj_subset['sentiment'] = doj_subset['contents'].apply(remove_named_ent_and_score)

# Print to view cleaned data 
print(doj_subset[['id', 'sentiment']])

            id  \
77     17-1235   
155    15-1522   
157     16-213   
162     16-381   
168     14-464   
...        ...   
13002   09-368   
13032   18-775   
13034   12-596   
13068   18-359   
13081  14-1377   

                                                             sentiment  
77       {'neg': 0.2, 'neu': 0.751, 'pos': 0.049, 'compound': -0.9931}  
155    {'neg': 0.129, 'neu': 0.804, 'pos': 0.066, 'compound': -0.9325}  
157     {'neg': 0.09, 'neu': 0.835, 'pos': 0.075, 'compound': -0.7579}  
162    {'neg': 0.121, 'neu': 0.798, 'pos': 0.081, 'compound': -0.9037}  
168    {'neg': 0.175, 'neu': 0.782, 'pos': 0.043, 'compound': -0.9864}  
...                                                                ...  
13002  {'neg': 0.159, 'neu': 0.778, 'pos': 0.062, 'compound': -0.9737}  
13032   {'neg': 0.081, 'neu': 0.825, 'pos': 0.095, 'compound': 0.5267}  
13034   {'neg': 0.15, 'neu': 0.758, 'pos': 0.092, 'compound': -0.9571}  
13068  {'neg': 0.133, 'neu': 0.745, 'pos': 0.122, 'co

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [26]:
# Part C

# parse the sentiment scores into seperate columns and add to doj_subset
doj_subset['negative'] = doj_subset['sentiment'].apply(lambda x: x['neg'])
doj_subset['positive'] = doj_subset['sentiment'].apply(lambda x: x['pos'])
doj_subset['neutral'] = doj_subset['sentiment'].apply(lambda x: x['neu'])
doj_subset['compound'] = doj_subset['sentiment'].apply(lambda x: x['compound'])

# create new dataframe with combined columns: doj_subtset_wscore
doj_subset_wscore = doj_subset[['id', 
                                'contents', 
                                'negative', 
                                'positive', 
                                'neutral', 
                                'compound',
                               'topics_clean']].sort_values(by='negative', ascending=False)
# print results
top_2_neg_press_releases = doj_subset_wscore.head(2)
print(top_2_neg_press_releases[['id', 'contents', 'negative']])

         id  \
329  14-248   
572  13-312   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [27]:
# Part D

# calculate mean compound sentiment score for each topic 
mean_compound_scores = doj_subset_wscore.groupby('topics_clean')['compound'].agg('mean')

print(mean_compound_scores)

topics_clean
Civil Rights             -0.087140
Hate Crimes              -0.935576
Project Safe Childhood   -0.653711
Name: compound, dtype: float64


In [28]:
# Part E

# The variation in sentiment scores is influenced by the nature of each topic, 
# with "Hate Crimes" often being more negative due to its association with harmful actions, 
# "Project Safe Childhood" leaning towards neutral to slightly positive sentiment due to child 
# safety initiatives, and "Civil Rights" displaying a range of sentiments reflecting discussions 
# about progress and challenges in civil rights.

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [34]:
# custom stopwords

custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [35]:
# Part A

# intialize SnowballStemmer in english
snowball_stemmer = SnowballStemmer("english")

# write function to preprocess text
def preprocess_text(raw_text):
    
    # convert all words to lowercase
    text = raw_text.lower()
    # split the words
    words = text.split()
    
    # Remove stopwords and custom_doj_stopwords
    words = [word for word in words if word not in (stopwords.words("english") + custom_doj_stopwords)]
    
    # Only retain alpha characters and words 4 characters or longer
    words = [word for word in words if word.isalpha() and len(word) >= 4]
    
    # apply SnowballStemmer()
    words = [snowball_stemmer.stem(word) for word in words]
    
    # put processed words back together
    processed_text = " ".join(words)
    
    return processed_text

In [36]:
# Part B

# apply the function to dataframe column "contents"
doj_subset_wscore['processed_text'] = doj_subset_wscore['contents'].apply(preprocess_text)

In [37]:
# Part C

# print results
print("Press Release for ID 16-718:")
print(doj_subset_wscore[doj_subset_wscore['id'] == '16-718'][['id', 'contents', 'processed_text']])

print("\nPress Release for ID 16-217:")
print(doj_subset_wscore[doj_subset_wscore['id'] == '16-217'][['id', 'contents', 'processed_text']])

Press Release for ID 16-718:
           id  \
11593  16-718   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [118]:
# use provided create_dtm function
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(),
        columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

# Part A

# create dtm using preprocessed releases with the required columns 
dtm_md = create_dtm(doj_subset_wscore['processed_text'],
                               doj_subset_wscore[['id', 'compound', 'topics_clean']])

In [119]:
# Function to get the top words for a specific subset
def get_top_words(subset_data, num_words=10):
    text_concat = ' '.join(subset_data['processed_text'])
    words = text_concat.split()
    word_counts = Counter(words)
    top_words = word_counts.most_common(num_words)
    return top_words


In [120]:
# Part B

# Calculate the top 5% quantile compound sentiment score
top_5_percentile = doj_subset_wscore['compound'].quantile(0.95)
top_sent_data = doj_subset_wscore[doj_subset_wscore['compound'] >= top_5_percentile]
top_words_pos = get_top_words(top_sent_data)

print("Top 10 words for press releases with the most positive sentiment:")
for word, count in top_words_pos:
    print(f"{word}: {count}")

Top 10 words for press releases with the most positive sentiment:
agreement: 138
enforc: 118
ensur: 110
state: 95
said: 78
settlement: 75
general: 74
feder: 72
communiti: 72
work: 71


In [40]:
# Part C

# Calculate the bottom 5% quantile compound sentiment score
bottom_5_percentile = doj_subset_wscore['compound'].quantile(0.05)
bottom_sent_data = doj_subset_wscore[doj_subset_wscore['compound'] <= bottom_5_percentile]
top_words_neg = get_top_words(bottom_sent_data)
print("\nTop 10 words for press releases with the most negative sentiment:")
for word, count in top_words_neg:
    print(f"{word}: {count}")



Top 10 words for press releases with the most negative sentiment:
crime: 130
offic: 121
assault: 116
hate: 110
defend: 109
feder: 90
sentenc: 88
prosecut: 87
victim: 84
guilti: 83


In [42]:
# Part D

# Print the top 10 words for press releases in each of the three topics_clean
topics = doj_subset_wscore['topics_clean'].unique()
for topic in topics:
    top_words_topic = get_top_words(doj_subset_wscore[doj_subset_wscore['topics_clean'] == topic])
    print(f"\nTop 10 words for press releases in '{topic}':")
    for word, count in top_words_topic:
        print(f"{word}: {count}")


Top 10 words for press releases in 'Hate Crimes':
prosecut: 458
crime: 445
hate: 421
sentenc: 420
feder: 418
charg: 406
said: 404
defend: 398
guilti: 374
year: 306

Top 10 words for press releases in 'Civil Rights':
hous: 588
offic: 504
enforc: 498
said: 490
discrimin: 476
feder: 459
violat: 444
alleg: 389
general: 386
state: 347

Top 10 words for press releases in 'Project Safe Childhood':
child: 995
exploit: 673
sexual: 570
safe: 475
project: 472
crimin: 404
prosecut: 357
sentenc: 329
children: 313
investig: 256


## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [121]:
selected_topics = ["Civil Rights", "Hate Crimes", "Project Safe Childhood"]
filtered_df = doj_subset_wscore[doj_subset_wscore['topics_clean'].isin(selected_topics)]

## Step 1: re-tokenize and store in list
text_raw_tokens = [wordpunct_tokenize(one_text) for one_text in 
                  filtered_df.processed_text]

## Step 2: use gensim create dictionary - gets all unique words across documents
text_raw_dict = corpora.Dictionary(text_raw_tokens)
raw_len = len(text_raw_dict) # get length for comparison below

## Step 3: filter out very rare and very common words (<5% or >95%)
lower_bound = round(doj_subset_wscore.processed_text.shape[0]*0.05)
upper_bound = round(doj_subset_wscore.processed_text.shape[0]*0.95)

### apply filtering to dictionary
text_raw_dict.filter_extremes(no_below = lower_bound,
                             no_above = upper_bound)

# demonstrates how many words were filtered out
print(f'Filtering out very rare and very common words reduced the \
length of dictionary from {str(raw_len)} to {str(len(text_raw_dict))}.')

## Step 4: apply dictionary to TOKENIZED texts
corpus_fromdict = [text_raw_dict.doc2bow(one_text) 
                   for one_text in text_raw_tokens]

Filtering out very rare and very common words reduced the length of dictionary from 6107 to 533.


In [122]:
## Step 5: Estimate the model
ldamod = gensim.models.ldamodel.LdaModel(corpus_fromdict, 
                                         num_topics = 3, 
                                         id2word=text_raw_dict, 
                                         passes=6, 
                                         alpha = 'auto',
                                         per_word_topics = True)

topics = ldamod.print_topics(num_words=15)
for topic in topics:
    words = [word.split("*")[1].strip(' "') for word in topic[1].split(" + ")]
    print(words)

['child', 'exploit', 'sexual', 'safe', 'project', 'crimin', 'prosecut', 'sentenc', 'children', 'investig', 'guilti', 'state', 'pornographi', 'offic', 'minor']
['prosecut', 'sentenc', 'feder', 'charg', 'said', 'defend', 'guilti', 'crime', 'investig', 'indict', 'hate', 'year', 'victim', 'violat', 'general']
['hous', 'discrimin', 'enforc', 'said', 'disabl', 'alleg', 'fair', 'requir', 'agreement', 'inform', 'ensur', 'violat', 'feder', 'court', 'general']


## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [123]:
# Part A

# List of topic probabilities
l=[ldamod.get_document_topics(item, minimum_probability=0) for item in corpus_fromdict]
### print result
l[0:5]

# Assert statement 
assert len(l) == len(doj_subset_wscore)

[[(0, 0.016583532), (1, 0.98211384), (2, 0.0013026842)],
 [(0, 0.0005858688), (1, 0.9987285), (2, 0.0006855945)],
 [(0, 0.00070700096), (1, 0.99846566), (2, 0.0008273456)],
 [(0, 0.0005524633), (1, 0.998801), (2, 0.00064650347)],
 [(0, 0.0004396547), (1, 0.9990459), (2, 0.00051449233)]]

In [124]:
# Part B

# Extract the highest probability topic index for each document
top_topics = [max(probabilities, key=lambda x: x[1]) for probabilities in l]

# Add the topic probabilities to the dataframe
doj_subset_wscore['topic_probabilities'] = l

# Create the top_topic column
doj_subset_wscore['top_topic'] = [topic[0] for topic in top_topics]


In [125]:
# Part C
# Group the data by 'topics_clean' and 'top_topic' to count the number of documents.
grouped = doj_subset_wscore.groupby(['topics_clean', 'top_topic']).size().reset_index(name='count')
grouped

# Then, we'll pivot the data to have topics as columns and use normalize to get percentages.
pivot_table = pd.pivot_table(grouped, values='count', index='topics_clean', columns='top_topic')
normalized_table = pivot_table.div(pivot_table.sum(axis=1), axis=0)

# Print the normalized table
print(normalized_table)


,topics_clean,top_topic,count
0,Civil Rights,1,104
1,Civil Rights,2,201
2,Hate Crimes,1,246
3,Project Safe Childhood,0,165
4,Project Safe Childhood,2,1


top_topic                      0         1         2
topics_clean                                        
Civil Rights                 NaN  0.340984  0.659016
Hate Crimes                  NaN  1.000000       NaN
Project Safe Childhood  0.993976       NaN  0.006024


# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [126]:
# Part A

# create function to process text and return string with its bigrams structured as directed 
def create_bigram_onedoc(processed_text):
    words = processed_text.split()
    bigrams = ['_'.join(pair) for pair in zip(words, words[1:])]
    
    bigram_text = ' '.join(bigrams)
    
    return bigram_text

# apply function
doj_subset_wscore['processed_text_bigrams'] = doj_subset_wscore['processed_text'].apply(create_bigram_onedoc)

# Part B

# print selected columns for press release with id 16-217
press_release_16_217 = doj_subset_wscore[doj_subset_wscore['id'] == '16-217']
print(press_release_16_217[['id', 'processed_text', 'processed_text_bigrams']])

          id  \
6727  16-217   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [130]:
# Part C
# create dtm_bigram with create_dtm function and processed_text_bigrams column
dtm_bigram = create_dtm(doj_subset_wscore['processed_text_bigrams'],
                        doj_subset_wscore[['id', 'topics_clean', 'compound']])

# Part D

#unigram matrix
print(dtm_md.shape)

#bigram matrix
print(dtm_bigram.shape)

# Part E

# modify get_top_words function with ngrams
def get_top_bigrams(subset_data, num_bigrams=10):
    text_concat = ' '.join(subset_data['processed_text'])
    words = text_concat.split()
    bigrams = list(ngrams(words, 2))
    bigram_counts = Counter(bigrams)
    top_bigrams = bigram_counts.most_common(num_bigrams)
    return top_bigrams

# Get and print the top 10 bigrams for each topic
topics = doj_subset_wscore['topics_clean'].unique()

for topic in topics:
    top_bigrams_topic = get_top_bigrams(doj_subset_wscore[doj_subset_wscore['topics_clean'] == topic])
    print(f"\nTop 10 bigrams for press releases in '{topic}':")
    for bigram, count in top_bigrams_topic:
        print(f"{bigram}: {count}")

(717, 6111)
(717, 63645)

Top 10 bigrams for press releases in 'Hate Crimes':
('hate', 'crime'): 299
('plead', 'guilti'): 269
('special', 'agent'): 117
('year', 'prison'): 104
('thoma', 'general'): 95
('grand', 'juri'): 93
('said', 'thoma'): 91
('act', 'general'): 85
('face', 'maximum'): 77
('feder', 'hate'): 76

Top 10 bigrams for press releases in 'Civil Rights':
('fair', 'hous'): 225
('princip', 'deputi'): 221
('deputi', 'general'): 221
('vanita', 'head'): 200
('general', 'vanita'): 199
('said', 'princip'): 186
('announc', 'today'): 116
('unit', 'state'): 106
('consent', 'decre'): 92
('act', 'general'): 80

Top 10 bigrams for press releases in 'Project Safe Childhood':
('project', 'safe'): 472
('child', 'exploit'): 266
('child', 'pornographi'): 237
('sexual', 'exploit'): 218
('plead', 'guilti'): 188
('child', 'sexual'): 177
('exploit', 'obscen'): 175
('obscen', 'section'): 161
('safe', 'childhood'): 161
('exploit', 'children'): 156


# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

,id,contents,negative,positive,neutral,compound,topics_clean,processed_text,topic_probabilities,top_topic,processed_text_bigrams
329,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.319,0.038,0.643,-0.9950,Hate Crimes,announc morn john made initi appear feder court crimin complaint charg hate crime charg relat threat made jewish woman own oper nosh jewish delicatessen bakeri arrest march base crimin complaint alleg interf feder protect threaten interf busi accord crimin alleg post threaten note vicin crimin complaint mere establish probabl presum innoc unless proven convict offens charg crimin face maximum statutori penalti year matter investig albuquerqu prosecut mark baker mexico aejean,"[(0, 0.016583532), (1, 0.98211384), (2, 0.0013026842)]",1,announc_morn morn_john john_made made_initi initi_appear appear_feder feder_court court_crimin crimin_complaint complaint_charg charg_hate hate_crime crime_charg charg_relat relat_threat threat_made made_jewish jewish_woman woman_own own_oper oper_nosh nosh_jewish jewish_delicatessen delicatessen_bakeri bakeri_arrest arrest_march march_base base_crimin crimin_complaint complaint_alleg alleg_interf interf_feder feder_protect protect_threaten threaten_interf interf_busi busi_accord accord_crimin crimin_alleg alleg_post post_threaten threaten_note note_vicin vicin_crimin crimin_complaint complaint_mere mere_establish establish_probabl probabl_presum presum_innoc innoc_unless unless_proven proven_convict convict_offens offens_charg charg_crimin crimin_face face_maximum maximum_statutori statutori_penalti penalti_year year_matter matter_investig investig_albuquerqu albuquerqu_prosecut prosecut_mark mark_baker baker_mexico mexico_aejean
572,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered mu

In [137]:
# Initialize the TF-IDF vectorizer to convert text to numerical vectors
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the processed_text
tfidf_matrix = tfidf_vectorizer.fit_transform(doj_subset_wscore['processed_text'])

# Calculate the cosine similarity between all pairs of press releases
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Step 4: Find the indices of the top pairs (excluding self-similarity)
n = len(doj_subset)
top_pairs = []
for i in range(n):
    for j in range(i + 1, n):
        top_pairs.append(((i, j), cosine_sim[i, j]))

# Sort the pairs by similarity and get the top two
top_pairs.sort(key=lambda x: x[1], reverse=True)
top_two_pairs = top_pairs[:2]

# Print the press releases for the top pairs
for pair, similarity in top_two_pairs:
    i, j = pair
    print(f"Similarity between Press Release {i} and Press Release {j}: {similarity:.4f}")
    print(f"Press Release {i}:\n{doj_subset_wscore.iloc[i]['processed_text']}\n")
    print(f"Press Release {j}:\n{doj_subset_wscore.iloc[j]['processed_text']}\n")

Similarity between Press Release 540 and Press Release 552: 0.9841
Press Release 540:
church resid sentenc today year prison follow lifetim term supervis releas entic minor engag sexual activ attempt transfer obscen materi announc act general kenneth blanco crimin act benjamin greenberg southern robert plead guilti march judg daniel hurley southern moor employ secret assign white hous time arrest remain custodi sinc moor sinc termin secret servic accord admiss made connect moor maintain profil social media applic provid platform exchang digit well voic text delawar state polic detect delawar child predat task forc creat profil pose moor engag number onlin chat mobil app includ moor number onlin chat moor undercov offic pose femal minor sexual natur sever moor sent pictur includ sexual explicit accord plea enforc discov moor communic minor moor admit sent sexual explicit imag entic minor send sexual explicit photo moor engag type behavior girl texa anoth girl moor request feder charg de

In [138]:
#The top two pairs of press releases seem to be related to similar crimes, 
# potentially different stages or aspects of the same type of crime, 
# but not necessarily the same case. The similarity scores indicate that 
# the content of these press releases has significant overlap, which 
# suggests they are covering similar topics.

# For example, in the first pair (Press Release 540 and Press Release 552), 
# both press releases involve individuals who pleaded guilty in federal court 
# for charges related to enticing minors into engaging in sexual activities and 
# attempting to transfer obscene materials. The press releases contain similar 
# details about the defendants, their activities, and their arrests.

# In the second pair (Press Release 178 and Press Release 187), the press releases are 
# related to the coercion of minors to engage in sexual acts and the distribution 
# of explicit content over the internet. Both press releases share common themes about 
# the nature of the crime, the guilty pleas, and the actions of the defendants.

# While these press releases may not be about the same exact case, they are certainly 
# closely related in terms of the crimes they describe. The high similarity scores 
# suggest that they share similar content and are likely reporting on similar types of 
# criminal activities.
